In [5]:
import gym
from gym.utils import play 
from gym import wrappers
from gym.wrappers import GrayScaleObservation, RecordEpisodeStatistics, TimeLimit, ResizeObservation, FrameStack

import matplotlib.pyplot as plt
import time
from hashlib import md5
import os

def apply_wrappers(env):
    env = GrayScaleObservation(env)
    env = ResizeObservation(env, (84, 84))
    env = FrameStack(env, num_stack=4)
    env = TimeLimit(env, 500)
    env = RecordEpisodeStatistics(env)
    return env

In [6]:
def DQN_learning(agent, episode_n = 100, t_max = 1000, visualize=False):
    # agent.epsilon_decrease = 1.0 / (0.75*episode_n)

    total_rewards = []
    for episode in range(episode_n):
        total_reward = 0
        if episode == 50 or episode == 200:
            agent.save_model()

        # print(agent.optimizer)
        
        state, info = env.reset()
        print(len(agent.tdrb))
        for t in range(t_max):
            action = agent.get_action(state)
            next_state, reward, done, truncated, info = env.step(action)
            total_reward += reward

            agent.add_sample(state, action, reward, next_state, done or truncated)
            agent.fit()

            if visualize:
                env.render()
    
            state = next_state
            if done or truncated:
                # print(info)
                break
                
        total_rewards.append(total_reward)
        
        print(f'episode: {episode}, total_reward: {info["episode"]["r"]},\
        episode length: {info["episode"]["l"]},\
        time: {info["episode"]["t"]}')
    
    return total_rewards


env = gym.make("ALE/Breakout-v5",  obs_type='rgb', frameskip=4)
env = apply_wrappers(env)

# env.metadata['render_fps'] = 30
episode_n = 500

frame_size = []
frame_size.extend(list(env.observation_space.shape))

agent = DQNAgent(frame_size, env.action_space.n, episode_n=episode_n)
#agent.load_model('/home/artem/atari_games/models/DQN_c11323c6ff7080195e86ee7b2e01d04b.pth')
# state, info = env.reset()
DQN_learning(agent, episode_n, visualize=False)
agent.save_model('/home/artem/atari_games/models/atari_model_1.pth')
#env.close()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


0


/tmp/ipykernel_3161/2570963249.py:139: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  obs = torch.tensor(obs, dtype=torch.float).squeeze().unsqueeze(dim=0)
/tmp/ipykernel_3161/2570963249.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs = torch.tensor(obs, dtype=torch.float)
/home/artem/atari_games/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


KeyboardInterrupt: 

#### Тестируем PrioritizedBuffer на LunarLander

In [4]:
torch.manual_seed(42)

def DQN_learning(agent, episode_n = 100, t_max = 1000):
    # agent.epsilon_decrease = 1.0 / (0.75*episode_n)

    total_rewards = []
    for episode in range(episode_n):
        total_reward = 0

        # print(agent.optimizer)
        
        state, info = env.reset()
        print(len(agent.tdrb))
        for t in range(t_max):
            action = agent.get_action(state)
            next_state, reward, done, truncated, info = env.step(action)
    
            total_reward += reward

            agent.add_sample(state, action, reward, next_state, done or truncated)
            agent.fit()
    
            state = next_state
    
            if done or truncated:
                # print(info)
                break
                
        total_rewards.append(total_reward)
        
        print(f'episode: {episode}, total_reward: {info["episode"]["r"]},\
        episode length: {info["episode"]["l"]},\
        time: {info["episode"]["t"]}')
    
    return total_rewards
    


env = gym.make('LunarLander-v2')
env = apply_wrappers(env)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

episode_n = 1000

agent = DQNAgent(state_dim, action_dim, episode_n=episode_n)

total_rewards = DQN_learning(agent, episode_n=episode_n)
agent.save_model()    

AssertionError: 

#### Visual evaluation

In [3]:
env = gym.make('LunarLander-v2', render_mode='human')
env = apply_wrappers(env)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

agent = DQNAgent(state_dim, action_dim)
agent.load_model('/home/artem/atari_games/models/DQN_da6ad5af96c552ca0ecc3e1d5325959d.pth')
agent.epsilon = 0.001

state, info = env.reset()

for t in range(1000):
    action = agent.get_action(state)
    next_state, reward, done, truncated, info = env.step(action)

    state = next_state

    env.render()

    if done or truncated:
        print(info)
        break

/home/artem/atari_games/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


{'episode': {'r': 281.86694, 'l': 235, 't': 9.974647}}


In [3]:
import numpy as np

import torch
from torch import nn

from tensordict import TensorDict

import torchrl
from torchrl.data.replay_buffers import PrioritizedReplayBuffer, ReplayBuffer, PrioritizedSampler
from torchrl.data import TensorDictPrioritizedReplayBuffer, LazyMemmapStorage, TensorDictReplayBuffer

class QFunction(nn.Module):
    def __init__(self, input_shape, action_n):
        super().__init__()
        self.input_shape = input_shape
        self.action_n = action_n

        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1),
            nn.ReLU()
        )

        self.conv_output_size = self._get_conv_output_size()

        self.model = nn.Sequential(
            self.conv_layers,
            nn.Flatten(),
            nn.Linear(self.conv_output_size, 512),
            nn.ReLU(),
            nn.Linear(512, action_n)
        )

    def _get_conv_output_size(self):
        out = self.conv_layers(torch.zeros(1, *self.input_shape))
        return np.prod(out.shape)

    def forward(self, input):
        return self.model(input)


class QFunction_baseline(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.model = nn.Sequential(
                nn.Linear(state_dim, 64),
                nn.ReLU(),
                nn.Linear(64, 64),
                nn.ReLU(),
                nn.Linear(64, action_dim)                
        )

    def forward(self, states):
        actions = self.model(states)
        return actions


class DQNAgent(nn.Module):
    def __init__(self, input_shape, action_n, lr=1e-3, gamma=0.99, batch_size=128, period=25, episode_n=1000):
        super().__init__()
        self.q_function = QFunction(input_shape, action_n)
        self.target_q_function = QFunction(input_shape, action_n)

        self.m = 5.0

        self.epsilon_min = 5e-3
        self.epsilon_decay = self.m * 1.0 / (episode_n)
        self.epsilon = 1.0 - 1.0 / action_n

        self.episode_n = episode_n
        self.current_episode = 0

        self.input_shape = input_shape
        self.action_n = action_n
        
        self.lr = lr
        self.gamma = gamma
        self.batch_size = batch_size
        self.period = period
        self.counter = 0
        
        self.optimizer = torch.optim.Adam(self.q_function.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[int(0.4 * self.episode_n), int(0.6 * self.episode_n), int(0.8*self.episode_n)], gamma=0.5)
        self.loss_f = nn.MSELoss()

        self.alpha = 0.8
        self.alpha_decay = self.m * 1.0 / episode_n
        self.beta = 0.3
        self.beta_decay = self.m * 1.0 / episode_n

        self.capacity = 400_000
        storage = LazyMemmapStorage(self.capacity, scratch_dir='/home/artem/atari_games/tmp/')
        self.sampler = PrioritizedSampler(self.capacity, alpha=self.alpha, beta=self.beta)
        self.tdrb = TensorDictReplayBuffer(storage=storage, sampler=self.sampler, priority_key='td_error')

    def save_model(self, path=f'/home/artem/atari_games/models/DQN_{md5(str(time.time()).encode()).hexdigest()}.pth'):
        state = {
            'model_dict': self.q_function.state_dict(),
            'optimizer_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon
        }
        torch.save(state, path)

    def load_model(self,path=f'/home/artem/atari_games/models/DQN_{md5(str(time.time()).encode()).hexdigest()}'):
        if os.path.exists(path):
            state = torch.load(path)
            self.optimizer.load_state_dict(state['optimizer_dict'])
            self.q_function.load_state_dict(state['model_dict'])
            self.target_q_function.load_state_dict(state['model_dict'])
            self.epsilon = state['epsilon']

    def decay_epsilon(self):
        self.epsilon = max((self.epsilon - self.epsilon_decay), self.epsilon_min)

    def decay_alpha(self):
        self.aplha = max(self.alpha - self.alpha_decay, 0.7)
        self.sampler._alpha = self.alpha

    def decay_beta(self):
        self.beta = min(self.beta + self.beta_decay, 0.5)
        self.sampler._beta = self.beta

    def decay_all(self):
        self.decay_epsilon()
        self.decay_alpha()
        self.decay_beta()

    def e_greedy_action(self, q_values):
        probs = np.ones(self.action_n) * self.epsilon / self.action_n
        probs[np.argmax(q_values.numpy())] += 1 - self.epsilon
        action = np.random.choice(np.arange(self.action_n), p=probs)
        return action
        
    def get_action(self, obs: np.ndarray) -> torch.Tensor:
        with torch.no_grad():
            obs = torch.tensor(obs, dtype=torch.float).squeeze().unsqueeze(dim=0)
            obs = torch.tensor(obs, dtype=torch.float)
            q_values = self.q_function(obs).squeeze()
            action = self.e_greedy_action(q_values)
            return action

    def add_sample(self, state, action, reward, next_state, done):
        if done:
            self.current_episode += 1
            self.scheduler.step()
            self.decay_all()
        self.tdrb.add(
            TensorDict({
                'state': torch.tensor(state, dtype=torch.float32).squeeze(),
                'action': torch.tensor(action),
                'reward': torch.tensor(reward),
                'next_state': torch.tensor(next_state, dtype=torch.float32).squeeze(),
                'done': torch.tensor(done)
                    }, batch_size=[])
        )
        # self.tdrb.add(
        #     TensorDict({
        #         'state': torch.tensor(state, dtype=torch.float32),
        #         'action': torch.tensor(action),
        #         'reward': torch.tensor(reward),
        #         'next_state': torch.tensor(next_state, dtype=torch.float32),
        #         'done': torch.tensor(done)
        #             }, batch_size=[])
        # )
    
    def fit(self):
        if self.batch_size < len(self.tdrb):
            # print('fit')
            if self.counter == self.period:
                self.counter = 0
                for parameter_freeze, parameter in zip(self.target_q_function.model.parameters(), self.q_function.model.parameters()):
                    with torch.no_grad():
                        parameter_freeze.data.copy_(parameter.data)
            else:
                self.counter += 1

            sample = self.tdrb.sample(self.batch_size)

            
            # target = sample['reward'] + self.gamma * (1 - torch.tensor(sample['done'], dtype=torch.uint8)) * \
            # (self.target_q_function(sample['next_state'])[torch.arange(self.batch_size), torch.argmax(self.q_function(sample['next_state']), dim=1)])
            # print(sample['next_state'].shape)
            # print(self.target_q_function(sample['next_state']).shape)
            # print(torch.argmax(self.q_function(sample['next_state']), dim=1).shape)
            target = sample['reward'] + self.gamma * (1 - torch.tensor(sample['done'], dtype=torch.uint8)) * \
            (self.target_q_function(sample['next_state'])[torch.arange(self.batch_size), torch.argmax(self.q_function(sample['next_state']), dim=1)])
            
            q_values = self.q_function(sample['state'])[torch.arange(self.batch_size), sample['action']]

            td_error = q_values - target.detach()
            sample['td_error'] = td_error.abs().detach()
            
            loss = torch.mean((td_error * sample['_weight'])**2)
            # loss = torch.mean((td_error)**2)
            self.tdrb.update_tensordict_priority(sample)
            
            loss.backward()
            # print('after update')
            self.optimizer.step()
            self.optimizer.zero_grad()
            # self.decay_all()

    
    

    


In [ ]:
agent = DQNAgent(state_dim, action_dim)

In [10]:
1 - torch.tensor(torch.tensor([True, False]), dtype=torch.uint8)

/tmp/ipykernel_20390/1352533337.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  1 - torch.tensor(torch.tensor([True, False]), dtype=torch.uint8)


tensor([0, 1], dtype=torch.uint8)

In [8]:
agent =  AtariAgent((1, 210, 160), 2)

v = torch.ones((210, 160), dtype=torch.uint8)

print(v.shape)
agent.get_action(v)

torch.Size([210, 160])


/tmp/ipykernel_20689/796179002.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs = torch.tensor(obs, dtype=torch.float).unsqueeze(dim=0).unsqueeze(dim=0)


1

In [6]:
t = torch.ones(1,1, 210, 160)
new_t = torch.flatten(t)
new_t.shape

f = nn.Flatten()
f(t).shape

torch.Size([1, 33600])

In [7]:
conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1),
            nn.ReLU()
        )

network = nn.Sequential(
            conv_layers,
            nn.Flatten(),
            nn.Linear(5632, 512)
)

network(t).shape

torch.Size([1, 512])

In [36]:
np.random.choice([0, 1] ,p=[0.9, 0.1])

0

In [46]:
t = torch.tensor([[1, 2, 3]]).numpy()
np.argmax(t)

2

In [47]:
torch.rand(3, 4)

tensor([[0.3359, 0.0023, 0.9459, 0.0583],
        [0.3100, 0.2059, 0.6849, 0.7648],
        [0.5246, 0.1823, 0.9524, 0.9177]])

In [13]:

rp = ReplayBuffer()
rp.add(1)
rp.add(2)
rp.extend([3, 4])
batch = rp.sample(batch_size=10)
batch

tensor([3, 4, 2, 2, 4, 1, 2, 4, 2, 3])

In [41]:
tdrp = TensorDictReplayBuffer()

dt0 = TensorDict({'a': torch.tensor([1]), 'b': torch.tensor([2])}, batch_size=[])
dt1 = TensorDict({'a': torch.rand(4, 1), 'b': torch.rand(4, 2)}, batch_size=[4])

tdrp.add(dt0)
tdrp.extend(dt1)
print(len(tdrp))
print(tdrp[0])

sample = tdrp.sample(batch_size=10)
sample['index']

5
TensorDict(
    fields={
        _data: TensorDict(
            fields={
                a: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False),
                b: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([]),
            device=None,
            is_shared=False),
        index: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([]),
    device=None,
    is_shared=False)


tensor([0, 2, 1, 4, 4, 3, 2, 4, 4, 0])

In [80]:
tdrp = TensorDictReplayBuffer(priority_key='td_error', sampler=PrioritizedSampler(max_capacity=4, alpha=100.0, beta=1.1))
tdrp.set_sampler(sampler=PrioritizedSampler(max_capacity=100, alpha=0.7, beta=0.5))
#dt0 = TensorDict({'a': torch.tensor([1]), 'b': torch.tensor([2])}, batch_size=[])
dt1 = TensorDict({'a': torch.rand(4, 1), 'b': torch.rand(4, 2)}, batch_size=[4])

#tdrp.add(dt0)
indeces = tdrp.extend(dt1)
print(indeces)
print(len(tdrp))
print(tdrp[0])

sample = tdrp.sample(batch_size=3)
print(sample['index'])
sample

tdrp.update_priority(indeces, priority=np.array([0.5, 0.8, 1.1, 1.2]))
sample = tdrp.sample(batch_size=3)
print(sample['index'])
#print(sample)
# tdrp.set_sampler(sampler=PrioritizedSampler(max_capacity=100, alpha=0.01, beta=1.1))
#sample = tdrp.sample(batch_size=10)
print(sample['_weight'])

tensor([0, 1, 2, 3])
4
TensorDict(
    fields={
        _data: TensorDict(
            fields={
                a: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
                b: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([]),
            device=None,
            is_shared=False),
        index: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([]),
    device=None,
    is_shared=False)
tensor([3, 0, 1])
tensor([0, 3, 2])
tensor([1.0000, 0.7361, 0.7588])


In [77]:
sample = tdrp.sample(batch_size=3)
print(sample['index'])
#print(sample)
# tdrp.set_sampler(sampler=PrioritizedSampler(max_capacity=100, alpha=0.01, beta=1.1))
#sample = tdrp.sample(batch_size=10)
print(sample["a"])

tensor([1, 3, 2])
tensor([[0.1765],
        [0.9584],
        [0.7441]])


In [81]:
dt2 = TensorDict({'a': torch.rand(1, 1), 'b': torch.rand(1, 2)}, batch_size=[1])
tdrp.add(dt2)

4

In [82]:
sample = tdrp.sample(batch_size=5)
print(sample['index'])
#print(sample)
# tdrp.set_sampler(sampler=PrioritizedSampler(max_capacity=100, alpha=0.01, beta=1.1))
#sample = tdrp.sample(batch_size=10)
print(sample["a"])

RuntimeError: stacking tensordicts requires them to have congruent batch sizes, got td1.batch_size=torch.Size([]) and td2.batch_size=torch.Size([1])

In [147]:
loss = (sample['a'] - torch.tensor([0.002, 1.0, 10.0]).reshape(3,1))**2

In [148]:
loss

tensor([[9.0494e-01],
        [9.6114e-03],
        [9.1567e+01]])

In [149]:
sample['td_error'] = loss.abs()

In [150]:
tdrp.update_tensordict_priority(sample)

In [164]:
sample = tdrp.sample(batch_size=3)
print(sample['index'])

tensor([3, 3, 3])


In [71]:
torch.tensor([0.002, 1.0, 10.0]).reshape(3,1)

tensor([[2.0000e-03],
        [1.0000e+00],
        [1.0000e+01]])

In [90]:
torch.arange(4).shape

torch.Size([4])

In [97]:
tdrp = TensorDictPrioritizedReplayBuffer(priority_key='td_error', alpha=0.8, beta=1.1)

#dt0 = TensorDict({'a': torch.tensor([1]), 'b': torch.tensor([2])}, batch_size=[])
dt1 = TensorDict({'a': torch.rand(4, 1), 'b': torch.rand(4, 2)}, batch_size=[4])

#tdrp.add(dt0)
indeces = tdrp.extend(dt1)
print(indeces)
print(len(tdrp))
print(tdrp[0])

sample = tdrp.sample(batch_size=10)
print(sample['index'])

tdrp.update_priority(indeces, priority=np.arange(3, -1, -1))
sample = tdrp.sample(batch_size=10)
print(sample['index'])
print(sample)

tensor([0, 1, 2, 3])
4
TensorDict(
    fields={
        _data: TensorDict(
            fields={
                a: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
                b: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([]),
            device=None,
            is_shared=False),
        index: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([]),
    device=None,
    is_shared=False)
tensor([2, 0, 1, 1, 3, 1, 1, 2, 0, 0])
tensor([2, 0, 1, 0, 0, 1, 2, 2, 1, 0])
TensorDict(
    fields={
        _weight: Tensor(shape=torch.Size([10]), device=cpu, dtype=torch.float32, is_shared=False),
        a: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        b: Tensor(shape=torch.Size([10, 2]), device=cpu, dtype=torch.float32, is_shared=False),
        index: Tensor(shape=torch.Size([10]), device

1.1